In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import hstack



In [2]:
from google.colab import files
def make_sub(predictions, filename='sub'):
  sub = pd.DataFrame({
      'uid': np.arange(len(predictions)),
      'target_ind': predictions 
  })
  sub.to_csv(filename + '.csv', index=False)
  files.download(filename + '.csv') 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
root = '/content/drive/MyDrive/Analytics Hack/'

train = pd.read_csv(root + "PS3_train.csv")
test = pd.read_csv(root + "PS3_test.csv")

In [5]:
train.head()

,content,title,uid,target_ind
0,Premium quality five pocket jean from Wrangler...,Amazon.com: Wrangler Men's Rugged Wear Relaxed...,B0000CBALT,247
1,If you're looking for a different kind of anim...,Sakura Diaries - Complete Series Collector's E...,B00005QFDT,453
2,"First things first: Yes, Thinking XXX features...",Thinking XXX (Extended Cut) (2006),B000BNXD50,228
3,Feathertouch. 100% Polyester Machine Wash Warm...,Amazon.com: Petite Feathertouch Pull-On Pant: ...,B0002LK9V2,223
4,"When you need outstanding fuel delivery, easy ...",ACDelco EP386 Fuel Pump,B000C9PA54,312


In [6]:
import re
def clean_text(string):
  out = re.sub(r'[^a-zA-Z0-9 ]', '', string)
  return out

train['content'] = train['content'].str.lower()
train['content'] = train['content'].apply(clean_text)
train['title'] = train['title'].str.lower()
train['title'] = train['title'].apply(clean_text)

test['content'] = test['content'].str.lower()
test['content'] = test['content'].apply(clean_text)
test['title'] = test['title'].str.lower()
test['title'] = test['title'].apply(clean_text)

In [7]:
train.target_ind.value_counts()

348    1243
351    1212
349     622
298     445
91      439
       ... 
14       35
137      35
377      35
19       35
170      35
Name: target_ind, Length: 500, dtype: int64

In [8]:
train.shape

(35112, 4)

In [9]:
test.shape

(8106, 3)

In [10]:
train.title[train.target_ind == 228]

2                           thinking xxx extended cut 2006
1069                                      plain dirty 2002
1289                            cathouse the specials 2010
1477                      eight millimeter8mm 2 rated 1999
2541                            nude showgirl championship
3785                  latin dimes reggaeton mix vol 1 2006
3929                      the ranch unrated and uncut 2004
5197                      eight millimeter8mm 2 rated 1999
5345                                  survival island 2006
5722                               gun a car a blonde 1997
5874                  rated x  a journey through porn 2005
5954     pornography  the secret history of civilisatio...
6618                                          gia vhs 1998
7262            hot springs hotel the complete series 1995
7934                                   bliss season 3 2004
8437                                                 speed
9343                                            flashpoi

In [11]:

# clf = CatBoostClassifier(iterations=60, learning_rate=0.2, max_depth=15, task_type="GPU")

In [12]:
df_t = train.sample(frac = 0.7).copy()
df_v = train.drop(df_t.index).copy()

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
# from catboost import CatBoostClassifier
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### TF-IDF Vectorization with MultinomialNB model

Inspiration: Haphazard model based on https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a


ngram_range: (1,2)
content_vec: 200,000
title_vec: 100,000
MultinomialNB with fit_prior: False

Test score: ~43.79%

Second place at ~42.96%

In [14]:
# stemmer = SnowballStemmer("english", ignore_stopwords=True)
stemmer = WordNetLemmatizer()
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.lemmatize(w) for w in analyzer(doc)])

In [15]:
# content_vec = StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=300_000)
# title_vec = StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=200_000)

# content = content_vec.fit_transform(df_t.content)
# title = title_vec.fit_transform(df_t.title)

# y_train =  df_t.target_ind
# X_train= hstack([content, title])

# model = MultinomialNB(fit_prior=False, alpha=0.005)
# model = model.fit(X_train, y_train)

In [16]:
# X_val = hstack([content_vec.transform(df_v.content), title_vec.transform(df_v.title)])
# pred_t = model.predict(X_train)
# pred_v = model.predict(X_val)

# print(f"Train acc = {np.mean(pred_t == df_t.target_ind)}")
# print(f"Validation acc = {np.mean(pred_v == df_v.target_ind)}")

In [19]:
# Tuning alpha
content_vec = StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=300_000)
title_vec = StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=200_000)

content = content_vec.fit_transform(train.content)
title = title_vec.fit_transform(train.title)

y_train =  train.target_ind
X_train= hstack([content, title])

clf = MultinomialNB(fit_prior=False)

nb_params = [{'alpha': [0.003, 0.004, 0.005, 0.006, 0.007]}]
clf = GridSearchCV(clf, nb_params, cv = 5, scoring='accuracy')
clf.fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_score_)

{'alpha': 0.005}
0.42851451225097437


With full data

In [15]:
content_vec = StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=400_000)
title_vec = StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=150_000)

content = content_vec.fit_transform(train.content)
title = title_vec.fit_transform(train.title)

y_train =  train.target_ind
X_train= hstack([content, title])

model = MultinomialNB(fit_prior=True, alpha=0.005)
model = model.fit(X_train, y_train)

In [16]:
pred_t = model.predict(X_train)

print(f"Train acc = {np.mean(pred_t == y_train)}")

Train acc = 0.7097288676236044


In [17]:
X_test = hstack([content_vec.transform(test.content), title_vec.transform(test.title)])

predictions = model.predict(X_test)
make_sub(predictions, filename='mnb_full_hypertuned')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Word2vec 

In [63]:
import gensim
import gensim.downloader as gensim_api

In [88]:
y_train = train.target_ind.to_numpy()

corpus = [] # list(train["content"])
corpus.extend(list(train["title"]))

## create list of lists of unigrams
lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)

## detect bigrams and trigrams
bigrams_detector = gensim.models.phrases.Phrases(lst_corpus, 
                 delimiter=" ".encode(), min_count=5, threshold=10)
bigrams_detector = gensim.models.phrases.Phraser(bigrams_detector)
trigrams_detector = gensim.models.phrases.Phrases(bigrams_detector[lst_corpus], 
            delimiter=" ".encode(), min_count=5, threshold=10)
trigrams_detector = gensim.models.phrases.Phraser(trigrams_detector)

In [109]:
# fit word2vec
nlp = gensim.models.word2vec.Word2Vec(lst_corpus, size=15,   
            window=8, min_count=1, sg=1, iter=30)

In [110]:
# for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

In [111]:
# tokenize text
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=100, padding="post", truncating="post")

In [112]:
# Same preparation for test set
corpus = test["title"]
## create list of n-grams
lst_corpus = []
for string in corpus:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, 
                 len(lst_words), 1)]
    lst_corpus.append(lst_grams)
    ## detect common bigrams and trigrams using the fitted detectors
lst_corpus = list(bigrams_detector[lst_corpus])
lst_corpus = list(trigrams_detector[lst_corpus])
## text to sequence with the fitted tokenizer
lst_text2seq = tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X_test = kprocessing.sequence.pad_sequences(lst_text2seq, maxlen=15,
             padding="post", truncating="post")

In [113]:
X_train.shape

(8106, 100)

In [114]:
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 15))
for word,idx in dic_vocabulary.items():
  ## update the row with vector
  try:
    embeddings[idx] =  nlp[word]
  ## if word not in model then skip and the row stays all 0s
  except:
    pass

<ipython-input-114-e05ce39095cf>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embeddings[idx] =  nlp[word]


In [115]:
embeddings.shape

(11612, 15)

In [116]:
word = "plain"
print("dic[word]:", dic_vocabulary[word], "|idx")
print("embeddings[idx]:", embeddings[dic_vocabulary[word]].shape, 
      "|vector")

dic[word]: 4905 |idx
embeddings[idx]: (15,) |vector


In [117]:
## input
x_in = layers.Input(shape=(15,))

## embedding
x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=15, trainable=False)(x_in)
                     
## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2, 
                         return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2))(x)

## final dense layers
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
y_out = layers.Dense(500, activation='softmax')(x)## compile
model = models.Model(x_in, y_out)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 15)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 15, 15)            174180    
                                                                 
 bidirectional_6 (Bidirectio  (None, 15, 30)           3720      
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 30)               5520      
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 64)                1984      
                                                                 
 dense_11 (Dense)            (None, 128)               8320

In [118]:
## encode y
dic_y_mapping = {n:label for n,label in 
                 enumerate(np.unique(y_train))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
y_train = np.array([inverse_dic[y] for y in y_train])

## train
training = model.fit(x=X_train, y=y_train, batch_size=256, 
                     epochs=10, shuffle=True, verbose=0, 
                     validation_split=0.3)

## plot loss and accuracy
metrics = [k for k in training.history.keys() if ("loss" not in k) and ("val" not in k)]
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(18,6))

ax[0].set(title="Training")
ax11 = ax[0].twinx()
ax[0].plot(training.history['loss'], color='black')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss', color='black')
for metric in metrics:
    ax11.plot(training.history[metric], label=metric)
ax11.set_ylabel("Score", color='steelblue')
ax11.legend()

ax[1].set(title="Validation")
ax22 = ax[1].twinx()
ax[1].plot(training.history['val_loss'], color='black')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss', color='black')
for metric in metrics:
     ax22.plot(training.history['val_'+metric], label=metric)
ax22.set_ylabel("Score", color="steelblue")
plt.show()

ValueError: ignored

In [ ]:
## test
predicted_prob = model.predict(X_test)
predicted = [dic_y_mapping[np.argmax(pred)] for pred in 
             predicted_prob]

### BERT

Take lite, too much time on CPU

In [25]:
#!pip install transformers

In [26]:
# import torch
# from tqdm.notebook import tqdm

# from transformers import BertTokenizer
# from torch.utils.data import TensorDataset

# from transformers import BertForSequenceClassification

In [18]:
# possible_labels = train.target_ind.unique()

# label_dict = {}
# for index, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = index
# label_dict

# train['label'] = train.target_ind.replace(label_dict)

In [27]:
# df_t = train.sample(frac = 0.7).copy()
# df_v = train.drop(df_t.index).copy()

In [20]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
#                                           do_lower_case=True)
                                          
# encoded_data_train = tokenizer.batch_encode_plus(
#     df_t.title.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt'
# )

# encoded_data_val = tokenizer.batch_encode_plus(
#     df_v.title.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt'
# )


# input_ids_train = encoded_data_train['input_ids']
# attention_masks_train = encoded_data_train['attention_mask']
# labels_train = torch.tensor(df_t.label.values)

# input_ids_val = encoded_data_val['input_ids']
# attention_masks_val = encoded_data_val['attention_mask']
# labels_val = torch.tensor(df_v.label.values)

# dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
# dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(train.label.values),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# batch_size = 4

# dataloader_train = DataLoader(dataset_train, 
#                               sampler=RandomSampler(dataset_train), 
#                               batch_size=batch_size)

# dataloader_validation = DataLoader(dataset_val, 
#                                    sampler=SequentialSampler(dataset_val), 
#                                    batch_size=batch_size)

In [23]:
# from transformers import AdamW, get_linear_schedule_with_warmup

# optimizer = AdamW(model.parameters(),
#                   lr=1e-5, 
#                   eps=1e-8)
                  
# epochs = 5

# scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                             num_warmup_steps=0,
#                                             num_training_steps=len(dataloader_train)*epochs)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
# import random

# seed_val = 42
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)

# def evaluate(dataloader_val):

#     model.eval()
    
#     loss_val_total = 0
#     predictions, true_vals = [], []
    
#     for batch in dataloader_val:
        
#         batch = tuple(b.to(device) for b in batch)
        
#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                   'labels':         batch[2],
#                  }

#         with torch.no_grad():        
#             outputs = model(**inputs)
            
#         loss = outputs[0]
#         logits = outputs[1]
#         loss_val_total += loss.item()

#         logits = logits.detach().cpu().numpy()
#         label_ids = inputs['labels'].cpu().numpy()
#         predictions.append(logits)
#         true_vals.append(label_ids)
    
#     loss_val_avg = loss_val_total/len(dataloader_val) 
    
#     predictions = np.concatenate(predictions, axis=0)
#     true_vals = np.concatenate(true_vals, axis=0)
            
#     return loss_val_avg, predictions, true_vals
    
# for epoch in tqdm(range(1, epochs+1)):
    
#     model.train()
    
#     loss_train_total = 0

#     progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
#     for batch in progress_bar:

#         model.zero_grad()
        
#         batch = tuple(b.to(device) for b in batch)
        
#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                   'labels':         batch[2],
#                  }       

#         outputs = model(**inputs)
        
#         loss = outputs[0]
#         loss_train_total += loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#         optimizer.step()
#         scheduler.step()
        
#         progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
#     torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
#     tqdm.write(f'\nEpoch {epoch}')
    
#     loss_train_avg = loss_train_total/len(dataloader_train)            
#     tqdm.write(f'Training loss: {loss_train_avg}')
    
#     val_loss, predictions, true_vals = evaluate(dataloader_validation)
#     tqdm.write(f'Validation loss: {val_loss}')


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/6145 [00:00<?, ?it/s]

KeyboardInterrupt: ignored